Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Landyn Francis"
COLLABORATORS = ""

---

## Differentiation options

1. Numerical differentiation
2. Symbolic differentiation
3. Automatic differentiation

    1. Forward mode differentiation
    2. Reverse mode differentiation

### 1. Numerical differentiation

### 2. Symbolic differentiation


### 3. Automatic differentiation

![image.png](attachment:83fb2c27-f8fd-4e05-a449-7231a3657819.png)

#### 3.A Forward mode 
Example:
$$ z = f(x_1, x_2) = x_1 x_2 + \sin(x_1)$$

In [13]:
import numpy as np
class ForwardDiff:
    def __init__(self,value, grad=0):
        self.value = value
        self.grad = grad
        
    def __add__(self,other):
        cls = type(self)
        other = other if isinstance(other, cls) else cls(other)
        v2 = self.value + other.value
        g2 = self.grad + other.grad
        return ForwardDiff(v2,g2)
    
    def __mul__(self, other):
        cls = type(self)
        other = other if isinstance(other, cls) else cls(other)
        out = cls(self.value * other.value, 
                    other.value * self.grad+ 
                     self.value * other.grad)
        return out
    
    def sin(self):
        cls = type(self)
        return cls(np.sin(self.value),
                     np.cos(self.value) * self.grad)
    
    def cos(self):
        cls = type(self)
        return cls(np.cos(self.value),
                    -np.sin(self.value) * self.grad)
    
    #If we want derivative with respect to x, we set the grad field of the x part to 0, and y to 1
x1 = ForwardDiff(2, grad=1) 
x2 = ForwardDiff(3, grad=0) 
f = x1*x2 + x1.sin()
f.value, f.grad
        

(6.909297426825682, 2.5838531634528574)

#### 3.B Reverse mode

Example:
$$ z = f(x_1, x_2) = x_1 x_2 + \sin(x_1)$$

In [1]:
import numpy as np
class ForwardDiff:
    def __init__(self, value, grad=None):
        self.value = value
        self.grad = np.zeros_like(value) if grad is None else grad
        
    
    def __add__(self, other):
        cls = type(self)
        other = other if isinstance(other, cls) else cls(other)
        out = cls(self.value + other.value, 
                    self.grad + other.grad)
        return out
    __radd__ = __add__
    
    def __repr__(self):
        return f"{self.__class__.__name__}(data={self.value}, grad={self.grad})"
x = ForwardDiff(2, 1)
y = ForwardDiff(3, 0)

f = x + y 
f

ForwardDiff(data=5, grad=1)

In [2]:
oldFD = ForwardDiff # Bad practice: do not do it
class ForwardDiff(oldFD):
    def __mul__(self, other):
        cls = type(self)
        other = other if isinstance(other, cls) else cls(other)
        out = cls(self.value * other.value, 
                    other.value * self.grad+ 
                     self.value * other.grad)
        return out
    
    __rmul__ = __mul__
    
x = ForwardDiff(2, 0)
y = ForwardDiff(3, 1)

f1 = x * y
f2 = 2*x + 3*y + x*y
f1, f2

(ForwardDiff(data=6, grad=2), ForwardDiff(data=19, grad=5))

In [3]:
oldFD = ForwardDiff # Bad practice: do not do it
class ForwardDiff(oldFD):
    def log(self):
        cls = type(self)
        return cls(np.log(self.value),
                     1/self.value * self.grad)
    def exp(self):
        cls = type(self)
        out_val = np.exp(self.value)
        return cls(out_val,
                     out_val * self.grad)
    
    def sin(self):
        cls = type(self)
        return cls(np.sin(self.value),
                     np.cos(self.value) * self.grad)
    
    def cos(self):
        cls = type(self)
        return cls(np.cos(self.value),
                    -np.sin(self.value) * self.grad)
    
    def __pow__(self, other):
        cls = type(self)
        other = other if isinstance(other, cls) else cls(other)
        return (self.log() * other).exp()

    def __neg__(self): # -self
        return self * -1

    def __sub__(self, other): # self - other
        return self + (-other)

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    

x = ForwardDiff(2, 1)
y = ForwardDiff(3, 0)

f = x**y
f

ForwardDiff(data=7.999999999999998, grad=11.999999999999996)

In [15]:
import numpy as np

# vector jacobian product
def add_deriv(a,b,grad):
    return (grad, grad)

def no_parent_deriv(grad):
    return (grad)

class ReverseDiff:
    def __init__(self, value, parents = (), deriv=no_parent_deriv):
        self.value = value
        self.parents = parents
        self.deriv = deriv
        self.grad = None
        
    def backward(self,grad):
        parent_value = [p.value for p in parents]
        parent_grads = self.deriv(*parent_values,grad)
        for p,g2 in zip(self.parents, parent_grads):
            p.backward(g2)
        
        
    def __add__(self,other):
        v2 = self.value + other.value
        return ReverseDiff(v2, parents=(self,other),deriv=add_deriv)
    
    def __repr__(self):
        cls = type(self)
        return f"{cls.__name__}(value={self.value}, parents={self.parents}"
        
x = ReverseDiff(2)
y = ReverseDiff(3)
f = x+y
f.backward(1)
x.grad, y.grad

NameError: name 'parents' is not defined

In [4]:
import numpy as np
def add_vjp(a, b, grad):
    return grad, grad


def no_parents_vjp(grad):
    return (grad,)

class ReverseDiff:
    def __init__(self, value, parents=(), op='', vjp=no_parents_vjp):
        self.value = value
        self.parents = parents
        self.op = op
        self.vjp = vjp
        self.grad = None

    def backward(self, grad):
        self.grad = grad
        op_args = [p.value for p in self.parents]
        grads = self.vjp(*op_args, grad)
        for g, p in zip(grads, self.parents):
            p.backward(g)
    
    def __add__(self, other):
        cls = type(self)
        other = other if isinstance(other, cls) else cls(other)
        out = cls(self.value + other.value, 
                  parents=(self, other),
                  op='+',
                  vjp=add_vjp)
        return out
    
    __radd__ = __add__
    
    def __repr__(self):
        cls = type(self)
        return f"{cls.__name__}(value={self.value}, parents={self.parents}, op={self.op}, grad={self.grad})"
    
x = ReverseDiff(2)
y = ReverseDiff(3)

f = x + y + 3
f.backward(1)
f
x.grad, y.grad

(1, 1)

In [5]:
oldRD = ReverseDiff # Bad practice: do not do it

def mul_vjp(a, b, grad):
    return grad * b, grad * a

class ReverseDiff(oldRD):
    def __mul__(self, other):
        cls = type(self)
        other = other if isinstance(other, cls) else cls(other)
        out = cls(self.value * other.value,
                 parents=(self, other),
                 op='*',
                 vjp=mul_vjp)
        return out
    
    __rmul__ = __mul__
    
x = ReverseDiff(2)
y = ReverseDiff(3)

f1 = 5*x + 7* y
f1.backward(1)
x.grad, y.grad

(5, 7)

In [6]:
f2 = x*y
f2.backward(1)
x.grad, y.grad

(3, 2)